In [1]:
res = {
    "class_instance": None
}

def class_decorator(cls):
   for name, method in cls.__dict__.iteritems():
        if hasattr(method, "use_class"):
            # do something with the method and class
            res["class_instance"] = cls
#             print(name, cls)
   return cls

def method_decorator(view):
    # mark the method as something that requires view's class
    view.use_class = True
    return view

@class_decorator
class ModelA(object):
    @method_decorator
    def a_method(self):
        # do some stuff
        print("a_method")
        pass
    
obj = ModelA()
print(res["class_instance"] == obj.__class__)
from pydoc import locate
a = locate("toto")
print(a)

True
None


In [3]:
from protocol0.components.SongDataManager import save_to_song_data, song_synchronizable_class
from protocol0.enums.BarLengthEnum import BarLengthEnum
from protocol0.enums.RecordTypeEnum import RecordTypeEnum
from protocol0.utils.utils import scroll_values

length = scroll_values(
            list(BarLengthEnum), BarLengthEnum.FOUR, True
        )
print(length)

8 bars
